## Notebook 1

In [80]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import SVC
from sklearn import metrics
import pickle

In [81]:
df = pd.read_csv("Sleep_health_and_lifestyle_dataset.csv", on_bad_lines='skip', sep=',')

In [82]:
df.head()

,Person ID,Gender,Age,Occupation,Sleep Duration,Quality of Sleep,Physical Activity Level,Stress Level,BMI Category,Blood Pressure,Heart Rate,Daily Steps,Sleep Disorder
0,1,Male,27,Software Engineer,6.1,6,42,6,Overweight,126/83,77,4200,NaN
1,2,Male,28,Doctor,6.2,6,60,8,Normal,125/80,75,10000,NaN
2,3,Male,28,Doctor,6.2,6,60,8,Normal,125/80,75,10000,NaN
3,4,Male,28,Sales Representative,5.9,4,30,8,Obese,140/90,85,3000,Sleep Apnea
4,5,Male,28,Sales Representative,5.9,4,30,8,Obese,140/90,85,3000,Sleep Apnea


In [83]:
print(df.isnull().sum())

Person ID                    0
Gender                       0
Age                          0
Occupation                   0
Sleep Duration               0
Quality of Sleep             0
Physical Activity Level      0
Stress Level                 0
BMI Category                 0
Blood Pressure               0
Heart Rate                   0
Daily Steps                  0
Sleep Disorder             219
dtype: int64


In [84]:
df['Sleep Disorder'] = df['Sleep Disorder'].fillna('No disorder')

In [85]:
df_copy = pd.concat([df, df['Blood Pressure'].str.split('/', expand=True)], axis=1).drop(
    'Blood Pressure', axis=1)

In [86]:
df_copy=df_copy.rename(columns={0: 'BloodPressure_high', 1: 'BloodPressure_low'})
df_copy['BloodPressure_high'] = df_copy['BloodPressure_high'].astype(float)
df_copy['BloodPressure_low'] = df_copy['BloodPressure_low'].astype(float)
df_copy

,Person ID,Gender,Age,Occupation,Sleep Duration,Quality of Sleep,Physical Activity Level,Stress Level,BMI Category,Heart Rate,Daily Steps,Sleep Disorder,BloodPressure_high,BloodPressure_low
0,1,Male,27,Software Engineer,6.1,6,42,6,Overweight,77,4200,No disorder,126.0,83.0
1,2,Male,28,Doctor,6.2,6,60,8,Normal,75,10000,No disorder,125.0,80.0
2,3,Male,28,Doctor,6.2,6,60,8,Normal,75,10000,No disorder,125.0,80.0
3,4,Male,28,Sales Representative,5.9,4,30,8,Obese,85,3000,Sleep Apnea,140.0,90.0
4,5,Male,28,Sales Representative,5.9,4,30,8,Obese,85,3000,Sleep Apnea,140.0,90.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
369,370,Female,59,Nurse,8.1,9,75,3,Overweight,68,7000,Sleep Apnea,140.0,95.0
370,371,Female,59,Nurse,8.0,9,75,3,Overweight,68,7000,Sleep Apnea,140.0,95.0
371,372,Female,59,Nurse,8.1,9,75,3,Overweight,68,7000,Sleep Apnea,140.0,95.0
372,373,Female,59,Nurse,8.1,9,75,3,Overweight,68,7000,Sleep Apnea,140.0,95.0


In [87]:
df_copy = df_copy.copy()
le_gender = LabelEncoder()
le_sleep = LabelEncoder()
df_copy['Gender'] = le_gender.fit_transform(df_copy['Gender'])
df_copy['Occupation'] = LabelEncoder().fit_transform(df_copy['Occupation'])
df_copy['BMI Category'] = LabelEncoder().fit_transform(df_copy['BMI Category'])
df_copy['Sleep Disorder'] = le_sleep.fit_transform(df_copy['Sleep Disorder'])

In [88]:
New_df_1 = df_copy[['Gender', 'Age', 'Physical Activity Level', 'Heart Rate', 'Daily Steps', 'BloodPressure_high', 'BloodPressure_low', 'Sleep Disorder']]
New_df_2 = df_copy[['Physical Activity Level', 'Heart Rate', 'Daily Steps', 'Sleep Disorder']]

In [89]:
X_1 = New_df_1.iloc[:,:-1] 
X_2 = New_df_2.iloc[:,:-1] 

y_1 = New_df_1['Sleep Disorder'] 
y_2 = New_df_2['Sleep Disorder'] 


In [90]:
svm = SVC()
svm_1 = svm.fit(X_1, y_1)
y_pred = svm_1.predict(X_1)
accuracy_test = metrics.accuracy_score(y_1, y_pred)
print(accuracy_test)

0.6577540106951871


In [91]:
svm = SVC()
svm_2 = svm.fit(X_2, y_2)
y_pred = svm_2.predict(X_2)
accuracy_test = metrics.accuracy_score(y_2, y_pred)
print(accuracy_test)

0.6577540106951871


In [92]:
model_pkl_file = "model_1.pkl"  

with open(model_pkl_file, 'wb') as file:  
    pickle.dump(svm_1, file)

In [93]:
model_pkl_file = "model_2.pkl"  

with open(model_pkl_file, 'wb') as file:  
    pickle.dump(svm_2, file)

In [94]:
model_pkl_file = "labelencoder.pkl"  

data_to_save = {
    "le_sleep": le_sleep,
    "le_gender": le_gender
}

with open(model_pkl_file, 'wb') as file:  
    pickle.dump(data_to_save, file)

In [95]:
with open(model_pkl_file, 'rb') as file:
    loaded_data = pickle.load(file)

le_sleep = loaded_data["le_sleep"]
le_gender = loaded_data["le_gender"]

In [99]:
X_1

,Gender,Age,Physical Activity Level,Heart Rate,Daily Steps,BloodPressure_high,BloodPressure_low
0,1,27,42,77,4200,126.0,83.0
1,1,28,60,75,10000,125.0,80.0
2,1,28,60,75,10000,125.0,80.0
3,1,28,30,85,3000,140.0,90.0
4,1,28,30,85,3000,140.0,90.0
...,...,...,...,...,...,...,...
369,0,59,75,68,7000,140.0,95.0
370,0,59,75,68,7000,140.0,95.0
371,0,59,75,68,7000,140.0,95.0
372,0,59,75,68,7000,140.0,95.0


In [98]:
y_2

0      1
1      1
2      1
3      2
4      2
      ..
369    2
370    2
371    2
372    2
373    2
Name: Sleep Disorder, Length: 374, dtype: int64